In [1]:
import os
####*IMPORANT*: Have to do this line *before* importing tensorflow
os.environ['PYTHONHASHSEED']=str(1)
import random
import sys

import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD,Adam,Adagrad,RMSprop
print("tensorflow version: ", tf.__version__)
print("python version: ", sys.version)
print("numpy version: ", np.__version__)

### WARN: Must have ###
# to make this notebook's output stable across runs
def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(1)
   tf.random.set_seed(1)
   np.random.seed(1)
   random.seed(1)
   os.environ['TF_DETERMINISTIC_OPS'] = '1'
   
reset_random_seeds()

# fashion MNIST 읽어 와서 신경망에 입력할 형태로 변환
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train.reshape(60000,784)
x_test = x_test.reshape(10000,784)
x_train=x_train.astype(np.float32)/255.0
x_test=x_test.astype(np.float32)/255.0
y_train=tf.keras.utils.to_categorical(y_train,10)
y_test=tf.keras.utils.to_categorical(y_test,10)

# 모델을 설계해주는 함수(모델을 나타내는 객체 model을 반환)
def run_model(x_train, y_train):    
    ####*IMPORANT*: Have to do this below line *every* re-run modeling & training
    reset_random_seeds()
    model=Sequential()
    model.add(Dense(512,activation='relu',input_shape=(784,)))
    model.add(Dense(512,activation='relu'))
    model.add(Dense(10,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(),
                  metrics=['accuracy'])
    model.fit(x_train, y_train, batch_size=256, epochs=5, verbose=0)
    return model

### Update: 22.04.01
n_of_models = 3
acc = 0
models = []
if tf.test.is_gpu_available():
  for n in range (0, n_of_models):
    models.append(run_model(x_train, y_train))
    acc += models[n].evaluate(x_test,y_test,verbose=0)[1]*100
    print('Model {}th accuracy: {}'.format(n, models[n].evaluate(x_test,y_test,verbose=0)[1]*100))
  acc /= n_of_models
else:
  n_of_models = 1
  n = 0
  models.append(run_model(x_train, y_train))
  print('Model {}th accuracy: {}'.format(n, models[n].evaluate(x_test,y_test,verbose=0)[1]*100))
  acc =  models[n].evaluate(x_test,y_test,verbose=0)[1]*100

g_org_model_acc = acc
print('Baseline (average) accuracy: {}'.format(acc))

models[0].summary()

# My Model
def run_proposed_model(x_train, y_train):  
    reset_random_seeds()  
    model = Sequential()
    model.add(Dense(1024,activation='relu',input_shape=(784,))) #number of nodes: 512 -> 1024
    model.add(Dense(512,activation='relu'))
    model.add(Dense(512,activation='relu')) #add layer
    model.add(Dense(10,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(),
                  metrics=['accuracy'])
    
    model.fit(x_train, y_train, batch_size=256, epochs=5, verbose=0)
    return model

n_of_models = 3
acc = 0
pro_models = []
if tf.test.is_gpu_available():
  for n in range (0, n_of_models):
    pro_models.append(run_proposed_model(x_train, y_train))
    acc += pro_models[n].evaluate(x_test,y_test,verbose=0)[1]*100 # verbose=1 when you want to see the progress
    print('Model {}th accuracy: {}'.format(n, 
                                           pro_models[n].evaluate(x_test,y_test,verbose=0)[1]*100))
  acc /= n_of_models
else:
  n_of_models = 1
  n = 0
  pro_models.append(run_proposed_model(x_train, y_train))
  print('Model {}th accuracy: {}'.format(n, 
                                         pro_models[n].evaluate(x_test,y_test,verbose=0)[1]*100))
  acc =  pro_models[n].evaluate(x_test,y_test,verbose=0)[1]*100 # verbose=1 when you want to see the progress

g_pro_model_acc = acc
print('Proposed (average) accuracy: {}'.format(acc))
  
# baseline vs Mine
print('Baseline vs. Mine: {} vs. {} under GPU {}'.format(g_org_model_acc, 
                                                         g_pro_model_acc, 
                                                         tf.test.is_gpu_available()))

tensorflow version:  2.8.2
python version:  3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
numpy version:  1.21.6
4431872/4422102 [==============================] - 0s 0us/step
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Model 0th accuracy: 86.94999814033508
Model 1th accuracy: 86.94999814033508
Model 2th accuracy: 86.94999814033508
Baseline (average) accuracy: 86.94999814033508
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 10)                5130      
                                                                 
Total params: 669,706
Train